# Create a heatmap to visualize population data
This is nice for showing things like disease prevalence, etc. For demonstration
purposes, we'll just show the rate of kids per 100k residents by ZCTA in VA.

In [ ]:
import os
import pandas as pd

import censusdata
import geopandas as gpd
import matplotlib.pyplot as plt
from us import states

In [ ]:
# Download total population by ZCTA in Virginia
geo = censusdata.censusgeo([('state', states.VA.fips), 
                            ('zip code tabulation area', '*')])
va_pop = censusdata.download(src='acs5', year=2019, geo=geo, var=['B01003_001E'])
va_pop['zcta'] = [x.geo[-1][1] for x in va_pop.index]
va_pop.set_index('zcta', inplace=True)
va_pop.columns = ['population']
va_pop.head()

In [ ]:
# Download ZCTA-level shape data from the census bureau.
URL = 'https://www2.census.gov/geo/tiger/TIGER2019/ZCTA5/tl_2019_us_zcta510.zip'
zcta_shape = gpd.read_file(URL)[['ZCTA5CE10', 'geometry']]
zcta_shape.set_index('ZCTA5CE10', inplace=True)
zcta_shape.head()

In [ ]:
df = va_pop.join(zcta_shape)

def geoplot(df, col, title, export=False):
    fig, ax = plt.subplots(figsize = (15,5))
    (gpd
        .GeoDataFrame(df)
        .plot(col, figsize=[12,9], legend=True, cmap="Blues", ax=ax))
    ax.set_title(title)
    plt.tight_layout()
    if export:
        plt.savefig(os.path.join('out',f'{col}.png'),format = 'png',
                    bbox_inches='tight')
    plt.show()

geoplot(df, "population", "Virginia population by ZCTA (2019)", export=True)

![Alt text](out/population.png)